In [1]:
%run 1-0mmyolo-common.ipynb

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Nov_30_19:08:53_PST_2020
Cuda compilation tools, release 11.2, V11.2.67
Build cuda_11.2.r11.2/compiler.29373293_0
gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Thu Feb 22 02:40:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                     

Extracting Dataset Version Zip to Forestfire2024-1 in coco-mmdetection:: 100%|██████████| 681/681 [00:05<00:00, 123.74it/s]


dataset classes: ['Alive Tree', 'Beetle-Fire Tree', 'Dead Tree', 'Debris']
dataset size: 471
dataset classes: ['Alive Tree', 'Beetle-Fire Tree', 'Dead Tree', 'Debris']
dataset size: 67


2024-02-22 02:40:47.159272: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-22 02:40:47.203972: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 02:40:47.203997: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 02:40:47.205059: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 02:40:47.212324: I tensorflow/core/platform/cpu_feature_guar

# Write custom Config file

In [ ]:
# %cd {HOME}/mmyolo
%cd {HOME}/mmdetection

In [1]:
model_folder="libra_rcnn"
base_model_name="libra-faster-rcnn_x101-64x4d_fpn_1x_coco"
custom_model_name=f"custom-{base_model_name}"
CUSTOM_CONFIG_PATH = f"{HOME}/mmdetection/configs/{model_folder}/{custom_model_name}.py"
print(CUSTOM_CONFIG_PATH)


CUSTOM_CONFIG = f"""
_base_ = './{base_model_name}.py'

# ========================Frequently modified parameters======================
# -----data related-----
model = dict(
    roi_head=dict(
        bbox_head=dict(num_classes={num_classes}),
))

dataset_type = 'COCODataset'
classes =  {classes}

data_root = '{dataset_location}' # Root directory of the dataset

train_ann_file = 'train/_annotations.coco.json'  # Annotation file for training set
train_data_prefix = 'train/'  # Prefix for training data directory

val_ann_file = 'valid/_annotations.coco.json'  # Annotation file for validation set
val_data_prefix = 'valid/'  # Prefix for validation data directory

class_name = {classes} 
num_classes = {num_classes}  # Number of classes in the dataset
metainfo = dict(classes=class_name, palette=[(20, 220, 60)])  # Metadata information for visualization

train_batch_size_per_gpu = {BATCH_SIZE}  # Batch size per GPU during training
#train_num_workers = 4  # Number of worker processes for data loading during training
persistent_workers = True  # Whether to use persistent workers during training

# -----train val related-----
# base_lr = 0.004  # Base learning rate for optimization
base_lr = 0.04
max_epochs = {MAX_EPOCHS}  # Maximum training epochs
num_epochs_stage2 = 20  # Number of epochs for stage 2 training

model_test_cfg = dict(
    multi_label=True,  # Multi-label configuration for multi-class prediction
    nms_pre=30000,  # Number of boxes before NMS
    score_thr=0.001,  # Score threshold to filter out boxes
    nms=dict(type='nms', iou_threshold=0.65),  # NMS type and threshold
    max_per_img=300)  # Maximum number of detections per image


# ========================Possible modified parameters========================
default_hooks = dict(
    checkpoint=dict(
        type="CheckpointHook",
        save_best="coco/bbox_mAP_50",
        rule="greater",
        max_keep_ckpts=10,
    ),
    early_stopping=dict(
        type="EarlyStoppingHook",
        monitor="coco/bbox_mAP_50",
        patience=15,
        min_delta=0.001
    ),
)

train_cfg=dict(
    max_epochs=max_epochs
)

data = dict(
    #samples_per_gpu=1,
    #workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        img_prefix='train/',
        classes=classes,
        ann_file='train/_annotations.coco.json.json'),
    val=dict(
        type=dataset_type,
        img_prefix='valid/',
        classes=classes,
        ann_file='valid/_annotations.coco.json'),
    test=dict(
        type=dataset_type,
        img_prefix='test/',
        classes=classes,
        ann_file='test/_annotations.coco.json'))

# -----data related-----
img_scale = (1024, 1024)  # width, height
# ratio range for random resize
random_resize_ratio_range = (0.1, 2.0)
# Cached images number in mosaic
mosaic_max_cached_images = 40
# Number of cached images in mixupep
mixup_max_cached_images = 20
# Batch size of a single GPU during validation
val_batch_size_per_gpu = 8
# Worker to pre-fetch data for each single GPU during validation
val_num_workers = 3

# Config of batch shapes. Only on val.
batch_shapes_cfg = dict(
    type='BatchShapePolicy',
    batch_size=val_batch_size_per_gpu,
    img_size=img_scale[0],
    size_divisor=32,
    extra_pad_ratio=0.5)

# -----train val related-----
lr_start_factor = 1.0e-5
dsl_topk = 13  # Number of bbox selected in each level
loss_cls_weight = 1.0
loss_bbox_weight = 2.0
qfl_beta = 2.0  # beta of QualityFocalLoss
weight_decay = 0.05

# Save model checkpoint and validation intervals
save_checkpoint_intervals = 10
# validation intervals in stage 2
val_interval_stage2 = 1
# The maximum checkpoints to keep.
max_keep_ckpts = 3
# single-scale training is recommended to
# be turned on, which can speed up training.
env_cfg = dict(cudnn_benchmark=True)
"""
with open(CUSTOM_CONFIG_PATH, 'w') as file:
    file.write(CUSTOM_CONFIG)

NameError: name 'HOME' is not defined

In [ ]:
## !python tools/train.py configs/ssd/costom_ssd512.py --resume | tee ../OUT_ssd.txt
!python tools/train.py configs/libra_rcnn/custom_libra-faster-rcnn_x101-64x4d_fpn_1x_coco.py | tee ../OUT_libra.txt

In [5]:
# Check Pytorch installation
import cv2
import os
import json
import roboflow
import random

import numpy as np

from mmdet.apis import init_detector, inference_detector

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Check MMDetection installation
from mmengine.utils import get_git_hash
from mmengine.utils.dl_utils import collect_env as collect_base_env

import mmdet

print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

import os
HOME = os.getcwd()
print("HOME:", HOME)

from roboflow import Roboflow
rf = Roboflow(api_key="a8fzziqukkQgsmGFnKgD")
project = rf.workspace("wildfire2024").project("forestfire2024")
dataset = project.version(1).download("coco-mmdetection")

CUSTOM_WEIGHTS_PATH = f"{HOME}/work_dirs/custom_libra-faster-rcnn_x101-64x4d_fpn_1x_coco/best_coco_bbox_mAP_50_epoch_199.pth"
CUSTOM_CONFIG_PATH = f"{HOME}/configs/libra_rcnn/custom_libra-faster-rcnn_x101-64x4d_fpn_1x_coco.py"
model = init_detector(CUSTOM_CONFIG_PATH, CUSTOM_WEIGHTS_PATH, device=DEVICE)

import supervision as sv

ds = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset.location}/test",
    annotations_path=f"{dataset.location}/test/_annotations.coco.json",
)

images = list(ds.images.values())

import numpy as np

# Load the dataset
ds = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset.location}/test",
    annotations_path=f"{dataset.location}/test/_annotations.coco.json",
)

# Define the search space for hyperparameters
param_space = {
    'CONFIDENCE_THRESHOLD': np.logspace(np.log10(0.001), 0, 20),  # Adjust the range as needed
    'NMS_IOU_THRESHOLD': np.logspace(np.log10(0.001), 0, 20),  # Adjust the range as needed
}

best_map = 0.0
best_params = {}

# Perform random search
for _ in range(20):  # You can adjust the number of random samples
    CONFIDENCE_THRESHOLD= np.random.choice(param_space['CONFIDENCE_THRESHOLD'])
    NMS_IOU_THRESHOLD= np.random.choice(param_space['NMS_IOU_THRESHOLD'])
    
    def callback(image: np.ndarray) -> sv.Detections:
        result = inference_detector(model, image)
        detections = sv.Detections.from_mmdetection(result)
        return detections[detections.confidence > CONFIDENCE_THRESHOLD].with_nms(threshold=NMS_IOU_THRESHOLD)

    mean_average_precision = sv.MeanAveragePrecision.benchmark(dataset=ds, callback=callback)

    if mean_average_precision.map50_95 > best_map:
        best_map = mean_average_precision.map50_95
        best_params = [CONFIDENCE_THRESHOLD,NMS_IOU_THRESHOLD]

print('Best mAP:', best_map)
print('Best Parameters:', best_params)

2.0.0+cu117 True
3.3.0
11.7
GCC 9.3
HOME: /work/van-speech-nlp/jindaznb/j-vis/mmdetection
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Forestfire2024-1 in coco-mmdetection:: 100%|██████████| 681/681 [00:06<00:00, 108.82it/s]


Loads checkpoint by local backend from path: /work/van-speech-nlp/jindaznb/j-vis/mmdetection/work_dirs/custom_libra-faster-rcnn_x101-64x4d_fpn_1x_coco/best_coco_bbox_mAP_50_epoch_199.pth
Best mAP: 0.4595606769745987
Best Parameters: [0.9526315789473684, 0.5736842105263158]
